In [1]:
import numpy as np
import cv2

In [2]:
# HSV_Values = [[4, 203, 175, 35, 255, 255],
#              [0, 155, 170, 178, 255, 255],
#              [132, 93, 104, 146, 180, 255]]

HSV_Values = [[0, 141, 175, 46, 255, 255],
             [0, 102, 170, 13, 255, 255],
             [132, 93, 104, 146, 180, 255]]

RGB_Values = [[50, 234, 255], 
             [51, 51, 255],
             [164, 63, 146]]

list_points = []

In [3]:
def cont(mask):
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0,0,0,0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        if area>500:
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x, y, w, h = cv2.boundingRect(approx)
            
    return x+w//2,y

In [4]:
def maskImg(image, HSV_Values, RGB_Values):
    
    imageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    temp_list = []
    count = 0
    
    for i in HSV_Values:
        lower = np.array(i[:3])
        upper = np.array(i[3:])
        mask = cv2.inRange(imageHSV, lower, upper)

        x, y = cont(mask)
        cv2.circle(result, (x, y),10, RGB_Values[count], cv2.FILLED)
        
        if x != 0 and y != 0:
            temp_list.append([x, y, count])
            
        count += 1
    return temp_list
            

In [5]:
def final_draw(list_points, RGB_Values):
    
    for j in list_points:
        cv2.circle(result, (j[0], j[1]),10, RGB_Values[j[2]], cv2.FILLED)

In [6]:
cap = cv2.VideoCapture(0)

while True:
    ret, image = cap.read()
    result = image.copy()
    temp_list = maskImg(image, HSV_Values, RGB_Values)
    
    if len(temp_list) != 0:
        for k in temp_list:
            list_points.append(k)
            
    if len(list_points) != 0:
        final_draw(list_points, RGB_Values)
        
    
    cv2.imshow("Image", result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()